This notebook performs profile normalization.
All profiles are normalized to the DMSO control treated profiles.

In [1]:
import argparse
import pathlib

import numpy as np
import pandas as pd
from pycytominer import normalize

# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")
try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False

In [2]:
if not in_notebook:
    argparser = argparse.ArgumentParser()
    argparser.add_argument(
        "--patient",
        type=str,
        required=True,
        help="Patient ID to process, e.g. 'P01'",
    )
    args = argparser.parse_args()
    patient = args.patient

else:
    patient = "SARCO361"

In [3]:
# pathing
sc_annotated_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/2.annotated_profiles/sc_anno.parquet"
).resolve(strict=True)
organoid_annotated_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/2.annotated_profiles/organoid_anno.parquet"
).resolve(strict=True)


# output path
sc_normalized_output_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/3.normalized_profiles/sc_norm.parquet"
).resolve()
organoid_normalized_output_path = pathlib.Path(
    f"{root_dir}/data/{patient}/image_based_profiles/3.normalized_profiles/organoid_norm.parquet"
).resolve()

organoid_normalized_output_path.parent.mkdir(parents=True, exist_ok=True)

In [4]:
# read in the data
sc_annotated_profiles = pd.read_parquet(sc_annotated_path)
organoid_annotated_profiles = pd.read_parquet(organoid_annotated_path)

### Normalize the single-cell profiles

In [5]:
sc_metadata_columns = [
    "patient",
    "object_id",
    "unit",
    "dose",
    "treatment",
    "image_set",
    "Well",
    "parent_organoid",
]
sc_features_columns = [
    col for col in sc_annotated_profiles.columns if col not in sc_metadata_columns
]

In [6]:
# find inf values and replace with NaN
sc_annotated_profiles[sc_features_columns] = sc_annotated_profiles[
    sc_features_columns
].replace([float("inf"), -float("inf")], np.nan)

In [7]:
# normalize the data
sc_normalized_profiles = normalize(
    sc_annotated_profiles,
    features=sc_features_columns,
    meta_features=sc_metadata_columns,
    method="standardize",
    samples="treatment == 'DMSO'",
)
sc_normalized_profiles.to_parquet(sc_normalized_output_path, index=False)
sc_normalized_profiles.head()

,patient,object_id,unit,dose,treatment,image_set,Well,parent_organoid,Colocalization_Nuclei_AGP.BF_MEAN.CORRELATION.COEFF,Colocalization_Nuclei_AGP.BF_MEDIAN.CORRELATION.COEFF,...,Area.Size.Shape_Cytoplasm_MIN.X,Area.Size.Shape_Cytoplasm_MAX.X,Area.Size.Shape_Cytoplasm_MIN.Y,Area.Size.Shape_Cytoplasm_MAX.Y,Area.Size.Shape_Cytoplasm_MIN.Z,Area.Size.Shape_Cytoplasm_MAX.Z,Area.Size.Shape_Cytoplasm_EXTENT,Area.Size.Shape_Cytoplasm_EULER.NUMBER,Area.Size.Shape_Cytoplasm_EQUIVALENT.DIAMETER,Area.Size.Shape_Cytoplasm_SURFACE.AREA
0,SARCO361,63,uM,1,Nilotinib,F6-4,F6,-1,1.128208,1.128208,...,-1.563725,-1.596488,0.795866,0.709358,-0.390267,-0.776504,1.447654,0.029134,0.083745,-0.517441
1,SARCO361,127,uM,1,Nilotinib,F6-4,F6,12,1.024314,1.024314,...,0.263549,0.292244,-0.943569,-0.941791,-0.390267,-0.776504,1.797869,-0.810362,0.584273,-0.246756
2,SARCO361,191,uM,1,Nilotinib,F6-4,F6,-1,-0.338469,-0.338469,...,-1.868271,-1.957520,1.121788,1.033182,-0.390267,-0.776504,0.303446,-0.250698,-0.393543,-0.464110
3,SARCO361,19,uM,1,Everolimus,C5-4,C5,45,1.394724,1.394724,...,0.544408,0.690048,0.243215,0.193374,-0.390267,1.737238,-1.005302,0.308966,0.214660,0.941926
4,SARCO361,39,uM,1,Everolimus,C5-4,C5,45,-1.291259,-1.291259,...,-0.057916,-0.078817,0.466401,0.403326,0.895252,7.344817,-1.477648,-3.888512,0.223567,3.051615


### Normalize the organoid profiles

In [8]:
organoid_annotated_profiles.head()

,patient,object_id,unit,dose,treatment,image_set,Well,single_cell_count,Colocalization_Organoid_AGP.BF_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MEDIAN.CORRELATION.COEFF,...,Area.Size.Shape_Organoid_MIN.X,Area.Size.Shape_Organoid_MAX.X,Area.Size.Shape_Organoid_MIN.Y,Area.Size.Shape_Organoid_MAX.Y,Area.Size.Shape_Organoid_MIN.Z,Area.Size.Shape_Organoid_MAX.Z,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Area.Size.Shape_Organoid_SURFACE.AREA
0,SARCO361,3,uM,1,Selumetinib,G10-7,G10,NaN,-0.051867,-0.051867,...,723.0,735.0,920.0,949.0,0.0,4.0,0.066092,7.0,5.600968,20.176451
1,SARCO361,12,uM,1,Nilotinib,F6-4,F6,1.0,-0.064466,-0.064466,...,534.0,1505.0,265.0,1099.0,2.0,16.0,0.000058,77.0,10.818614,233.427032
2,SARCO361,3,uM,10,Binimetinib,C8-7,C8,NaN,-0.009716,-0.009716,...,798.0,976.0,578.0,701.0,0.0,4.0,0.004887,56.0,9.350074,135.764496
3,SARCO361,45,uM,1,Everolimus,C5-4,C5,12.0,-0.073847,-0.073847,...,322.0,1196.0,361.0,1505.0,0.0,59.0,0.178001,4.0,271.688354,31089.646484
4,SARCO361,5,uM,10,Trametinib,F10-3,F10,NaN,0.055337,0.055337,...,1020.0,1343.0,1354.0,1539.0,15.0,21.0,0.701991,1.0,78.334473,365.284912


In [9]:
organoid_metadata_columns = [
    "patient",
    "object_id",
    "unit",
    "dose",
    "treatment",
    "image_set",
    "Well",
    "single_cell_count",
]
organoid_features_columns = [
    col
    for col in organoid_annotated_profiles.columns
    if col not in organoid_metadata_columns
]
# normalize the data
organoid_normalized_profiles = normalize(
    organoid_annotated_profiles,
    features=organoid_features_columns,
    meta_features=organoid_metadata_columns,
    method="standardize",
    samples="treatment == 'DMSO'",
)
organoid_normalized_profiles.to_parquet(organoid_normalized_output_path, index=False)
organoid_normalized_profiles.head()

,patient,object_id,unit,dose,treatment,image_set,Well,single_cell_count,Colocalization_Organoid_AGP.BF_MEAN.CORRELATION.COEFF,Colocalization_Organoid_AGP.BF_MEDIAN.CORRELATION.COEFF,...,Area.Size.Shape_Organoid_MIN.X,Area.Size.Shape_Organoid_MAX.X,Area.Size.Shape_Organoid_MIN.Y,Area.Size.Shape_Organoid_MAX.Y,Area.Size.Shape_Organoid_MIN.Z,Area.Size.Shape_Organoid_MAX.Z,Area.Size.Shape_Organoid_EXTENT,Area.Size.Shape_Organoid_EULER.NUMBER,Area.Size.Shape_Organoid_EQUIVALENT.DIAMETER,Area.Size.Shape_Organoid_SURFACE.AREA
0,SARCO361,3,uM,1,Selumetinib,G10-7,G10,NaN,-0.558121,-0.558121,...,0.940022,-2.848617,1.933864,-1.101632,-0.500376,-2.090429,-0.848919,-0.187277,-2.115236,-1.168251
1,SARCO361,12,uM,1,Nilotinib,F6-4,F6,1.0,-0.768010,-0.768010,...,0.420956,0.918519,-0.178544,-0.569424,0.005380,-0.549250,-1.243540,3.975833,-2.047617,-1.145283
2,SARCO361,3,uM,10,Binimetinib,C8-7,C8,NaN,0.144085,0.144085,...,1.146001,-1.669552,0.830897,-1.981549,-0.500376,-2.090429,-1.214684,2.726900,-2.066649,-1.155802
3,SARCO361,45,uM,1,Everolimus,C5-4,C5,12.0,-0.924303,-0.924303,...,-0.161277,-0.593228,0.131061,0.871084,-0.500376,4.973309,-0.180141,-0.365696,1.333158,2.178146
4,SARCO361,5,uM,10,Trametinib,F10-3,F10,NaN,1.227839,1.227839,...,1.755697,0.125953,3.333536,0.991718,3.292798,0.092908,2.951274,-0.544115,-1.172637,-1.131081
